In [7]:
import warnings
warnings.filterwarnings('ignore')
import os, sys
src_path = os.path.join(os.path.abspath('..'), 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

#from src.local_models.llm import MyLLM, LlamaCPPLLM
from llama_index.llms.openai import OpenAI
from local_models.embeddings import get_embed_model

from data_loader.splitting import split_by_md_headers
from data_loader.parsing import get_html, extract_md_tables

from llama_index.core import Document
from data_loader.chunking import chunk_docs_standalone
from llama_index.core import VectorStoreIndex
from llama_index.core import PromptTemplate

from data_loader.load_from_dir import rebuild_index
from utils import load_prompt



from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.agent.openai import OpenAIAgent


from dotenv import load_dotenv
import os

In [9]:
load_dotenv(override=True)

True

In [10]:
embed_model = get_embed_model(model_name=os.environ['embed_path'],  model_kwargs={'device': 'cpu'}, encode_kwargs = {'normalize_embeddings': True})
llm = OpenAI()

In [11]:
#first-time build

df = split_by_md_headers('../data/RAG-塞尔达王国之泪材料.md')
df['text_html'] = df['content'].apply(lambda x: get_html(x))
df = extract_md_tables(df)

In [12]:
df

,Header 1,Header 2,Header 3,Header 4,content,is_table,text,table
0,塞尔达王国之泪,概述,NaN,NaN,《塞尔达传说：王国之泪》是由任天堂企划制作本部开发的动作冒险游戏，作为《塞尔达传说：旷野之息...,0,《塞尔达传说：王国之泪》是由任天堂企划制作本部开发的动作冒险游戏，作为《塞尔达传说：旷野之息...,
1,塞尔达王国之泪,人物介绍,**林克**,NaN,主角 \n于「旷野之息」中击退大灾祸的勇者。在调查海拉鲁城堡的地下时，被卷入一场天地异变当...,0,主角 \n\n于「旷野之息」中击退大灾祸的勇者。在调查海拉鲁城堡的地下时，被卷入一场天地异...,
2,塞尔达王国之泪,人物介绍,**塞尔达**,NaN,海拉鲁王国公主 \n「塞尔达传说」系列的女主角，是寄宿著神圣力量的海拉鲁王国公主，拥有代代...,0,海拉鲁王国公主 \n\n「塞尔达传说」系列的女主角，是寄宿著神圣力量的海拉鲁王国公主，拥有...,
3,塞尔达王国之泪,人物介绍,**魔王加侬多夫**,NaN,魔王 \n自封印中苏醒之徒，在很久很久以前，海拉鲁建国时代出现的邪恶存在。掌握著「力量三角...,0,魔王 \n\n自封印中苏醒之徒，在很久很久以前，海拉鲁建国时代出现的邪恶存在。掌握著「力量...,
4,塞尔达王国之泪,人物介绍,**劳鲁**,NaN,劳鲁是一位非常重要的角色。他不仅是海拉鲁王国的首任国王，也是海底世界的守护者，同时还是玩家探...,0,劳鲁是一位非常重要的角色。他不仅是海拉鲁王国的首任国王，也是海底世界的守护者，同时还是玩家探...,
5,塞尔达王国之泪,人物介绍,**米涅鲁**,NaN,魂之贤者 \n劳鲁的亲姐姐，在遥远的神话时代与他一同和魔王交战的魂之贤者。 \n虽然在战...,0,魂之贤者 \n\n劳鲁的亲姐姐，在遥远的神话时代与他一同和魔王交战的魂之贤者。 \n\n...,
6,塞尔达王国之泪,人物介绍,**可盖大人**,NaN,回归的依盖队队长 \n与海拉鲁王国敌对的组织——依盖队的头目。因为其自身的存在感和莫名的威...,0,回归的依盖队队长 \n\n与海拉鲁王国敌对的组织——依盖队的头目。因为其自身的存在感和莫名...,
7,塞尔达王国之泪,人物介绍,**塔乌洛**,NaN,左纳乌调查队的队长 \n他对左纳乌文明的探究欲，整个海拉鲁无出其右，因重大发现而兴奋时会不...,0,左纳乌调查队的队长 \n\n他对左纳乌文明的探究欲，整个海拉鲁无出其右，因重大发现而兴奋时...,
8,塞尔达王国之泪,人物介绍,**普尔亚**,NaN,监视堡垒的指挥者 \n研究古代技术的权威第一人。年轻时就成为了海拉鲁王国的直属研究员，最近...,0,监视堡垒的指挥者 \n\n研究古代技术的权威第一人。年轻时就成为了海拉鲁王国的直属研究员，...,
9,塞尔达王国之泪,人物介绍,**洛贝利**,NaN,普尔亚平板的技术负责人 \n外表与众不同，但身为技术人员的本领可是超一流。 \n阿卡莱古...,0,普尔亚平板的技术负责人 \n\n外表与众不同，但身为技术人员的本领可是超一流。 \n\n...,


In [ ]:


table_docs = [Document(text=tbl, metadata={}) for tbl in df[df['is_table']==1]['table'].values]

text_docs = [Document(text=tbl, metadata={}) for tbl in df[df['is_table']==0]['text'].values]
text_nodes = [chunk_docs_standalone(text_docs[i], chunk_size=512, chunk_overlap=50)[0] for i in range(len(text_docs))]

table_index = VectorStoreIndex.from_documents(table_docs)
text_index = VectorStoreIndex(text_nodes)

#persist to disk
table_index.storage_context.persist(persist_dir="db_stores/table_index")
text_index.storage_context.persist(persist_dir='db_stores/text_index')


In [ ]:
#rebuild storage context from disk
table_index = rebuild_index("../db_stores/table_index")
text_index = rebuild_index("../db_stores/text_index")


In [17]:
table_engine = table_index.as_query_engine()
text_engine = text_index.as_query_engine()


In [18]:
# define tools
query_engine_tools = [
    QueryEngineTool(
        query_engine=table_engine,
        metadata=ToolMetadata(
            name="table_tool",
            description=(
                "Useful for retriving specific context from tables"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=text_engine,
        metadata=ToolMetadata(
            name="text_tool",
            description=(
                "Useful for retriving specific context from plain text"
            ),
        ),
    ),
]

In [19]:
# build agent
agent = OpenAIAgent.from_tools(
    query_engine_tools,
    max_function_calls=3,
    verbose=True,
    system_prompt=f"""\
        Respond in Chinese.
        
        If you inquire general-purpose questions, refer to the text_tool as a priority.
        For questions involving numbers and figures, refer to the table_tool first.
        If you need to analyze problems demanding a higher degree of rigorous reasoning, such as step-by-step instructions, 
        please utilize both the text_tool and table_tool to synthesize your answer.
        
        You must ALWAYS use at least one of the tools provided when answering a question; do NOT rely on prior knowledge.\
        """,
)

In [20]:
agent.query('王国之泪的背景设定是什么')

Added user message to memory: 王国之泪的背景设定是什么
=== Calling Function ===
Calling function: text_tool with args: {"input":"王国之泪的背景设定是什么"}
Got output: 王国之泪的背景设定在一个名为“王国之泪”的神秘大陆，这片土地曾经是一个繁荣的王国，但随着时间的流逝，王国陷入了衰败和遗忘。



Response(response='王国之泪的背景设定是在一个名为“王国之泪”的神秘大陆，这片土地曾经是一个繁荣的王国，但随着时间的流逝，王国陷入了衰败和遗忘。', source_nodes=[NodeWithScore(node=TextNode(id_='2b6c6b8b-9d7e-49ec-832c-c17b460accea', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ed8c2c92-76c2-4eb4-90e2-c2dbb46b46d5', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='e7021eb2ec88f4dead509e5c01af886d68ffa0627d4dd6e9bc146b05c0fc8416'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='50ac2445-8c18-46d1-b654-21cfcd061910', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='d9fa9bb2a0188a3cb73e754895c2998283f458ce338c8a3e3f78466236f3dea6')}, text='《塞尔达传说：王国之泪》是由任天堂企划制作本部开发的动作冒险游戏，作为《塞尔达传说：旷野之息》的正统续篇，该游戏在2019年6月12日通过网络直播节目“任天堂直面会: E3 2019”公开，并于2023年5月12日在Nintendo Switch平台上发售。  \n\n《塞尔达传说：王国之泪》继承了系列一贯的开放世界探索和解谜元素，玩家将再次扮演主角林克（Link），踏上一段全新的冒险之旅。游戏的故事背景设定在一个名为“王国之泪”的神秘大陆，这片土地上曾经有一个繁荣的王国，但随着时间的流逝，王国陷入了衰败和遗忘。  \n\n在这次冒险中，林克将探索广阔

In [21]:
agent.query('告诉我关于主角的一些事情')

Added user message to memory: 告诉我关于主角的一些事情
=== Calling Function ===
Calling function: text_tool with args: {"input":"主角"}
Got output: The hero who defeated a great calamity in "Breath of the Wild". While investigating the underground of Hyrule Castle, he became involved in a great upheaval. He will use the new power gained in his right hand to once again confront the anomaly invading Hyrule.



Response(response='在《荒野之息》中，主角是一个击败了巨大灾难的英雄。在调查海拉鲁城堡地下时，他卷入了一场巨大的动荡。他将利用右手获得的新力量，再次面对侵袭海拉鲁的异常现象。', source_nodes=[NodeWithScore(node=TextNode(id_='643dad6a-29a4-49e9-a210-e8ff18e296fc', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='e56937aa-a97f-49d4-88e6-92642d8c912f', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='da57e8d72bbde33c071a6c8adc6019342a3a6b6f862405a14c2846bb7983af81')}, text='主角  \n\n于「旷野之息」中击退大灾祸的勇者。在调查海拉鲁城堡的地下时，被卷入一场天地异变当中。他将驱使右手获得的新力量，再次挺身对抗侵袭海拉鲁的异变\n。', start_char_idx=0, end_char_idx=77, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.8391542463187526), NodeWithScore(node=TextNode(id_='e6c9aa22-eea6-4118-b070-630c5791bf5d', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo